<a href="https://colab.research.google.com/github/enesemretas/hingeprot-colab-ui/blob/main/notebooks/HingeProt_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# @title
%%bash
cd /content
rm -rf hingeprot-colab-ui
git clone https://github.com/enesemretas/hingeprot-colab-ui.git
cd hingeprot-colab-ui

apt-get update -qq
apt-get install -y -qq perl libblas3 dos2unix

# install libg2c.so.0 (legacy g77 runtime)
wget -q https://old-releases.ubuntu.com/ubuntu/pool/universe/g/gcc-3.4/gcc-3.4-base_3.4.6-6ubuntu3_amd64.deb
wget -q https://old-releases.ubuntu.com/ubuntu/pool/universe/g/gcc-3.4/libg2c0_3.4.6-6ubuntu3_amd64.deb
dpkg -i gcc-3.4-base_3.4.6-6ubuntu3_amd64.deb libg2c0_3.4.6-6ubuntu3_amd64.deb || true
apt-get -y -qq -f install
ldconfig

chmod +x hingeprot/* || true
dos2unix hingeprot/runHingeProt.pl 2>/dev/null || true

echo "Done setup."


(Reading database ... 117639 files and directories currently installed.)
Preparing to unpack gcc-3.4-base_3.4.6-6ubuntu3_amd64.deb ...
Unpacking gcc-3.4-base (3.4.6-6ubuntu3) over (3.4.6-6ubuntu3) ...
Preparing to unpack libg2c0_3.4.6-6ubuntu3_amd64.deb ...
Unpacking libg2c0 (1:3.4.6-6ubuntu3) over (1:3.4.6-6ubuntu3) ...
Setting up gcc-3.4-base (3.4.6-6ubuntu3) ...
Setting up libg2c0 (1:3.4.6-6ubuntu3) ...
Processing triggers for libc-bin (2.35-0ubuntu3.11) ...
Done setup.


Cloning into 'hingeprot-colab-ui'...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symboli

In [15]:
# @title HingeProt Colab UI (Radio input + One-click upload, no extra Colab upload button)
from google.colab import output, files
output.enable_custom_widget_manager()

import os, re, shutil, subprocess, zipfile, datetime, base64
import requests
import ipywidgets as W
from IPython.display import display, clear_output

# ---------- paths ----------
REPO_ROOT = "/content/hingeprot-colab-ui"
HINGEPROT_DIR = os.path.join(REPO_ROOT, "hingeprot")
RUN_PL = os.path.join(HINGEPROT_DIR, "runHingeProt.pl")
RUNS_ROOT = "/content/hingeprot_runs"
os.makedirs(RUNS_ROOT, exist_ok=True)

# ---------- helpers ----------
def _fetch_pdb(pdb_code: str, out_path: str):
    code = pdb_code.strip().upper()
    if not re.fullmatch(r"[0-9A-Z]{4}", code):
        raise ValueError("PDB code must be 4 characters (e.g., 4CLN).")
    url = f"https://files.rcsb.org/download/{code}.pdb"
    r = requests.get(url, timeout=30)
    if r.status_code != 200 or len(r.text) < 200:
        raise RuntimeError(f"Failed to fetch PDB {code} (HTTP {r.status_code}).")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(r.text)

def _detect_chains(pdb_path: str):
    chains = set()
    with open(pdb_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if line.startswith(("ATOM  ", "HETATM")) and len(line) > 21:
                ch = line[21].strip()
                if ch:
                    chains.add(ch)
    return sorted(chains)

def _safe_rm(path: str):
    if os.path.isdir(path):
        shutil.rmtree(path, ignore_errors=True)
    elif os.path.exists(path):
        try:
            os.remove(path)
        except:
            pass

def _zip_folder(folder_path: str, zip_path: str):
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
        for root, _, files_ in os.walk(folder_path):
            for fn in files_:
                full = os.path.join(root, fn)
                rel = os.path.relpath(full, folder_path)
                z.write(full, rel)

def _read_small_text(path: str, max_chars=6000):
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read(max_chars)
    except:
        return ""

def _run_hingeprot_one_chain(pdb_abs: str, chain: str, log_print):
    """
    Robust runner:
    - Copies PDB into HINGEPROT_DIR
    - Calls runHingeProt.pl with RELATIVE pdb filename
    """
    pdb_base = os.path.basename(pdb_abs)                 # e.g., input.pdb
    local_pdb = os.path.join(HINGEPROT_DIR, pdb_base)    # hingeprot/input.pdb
    shutil.copy2(pdb_abs, local_pdb)

    out_dir_name = f"{pdb_base}.{chain}"                 # input.pdb.A
    out_dir_in_hp = os.path.join(HINGEPROT_DIR, out_dir_name)
    _safe_rm(out_dir_in_hp)

    cmd = ["perl", RUN_PL, pdb_base, chain]              # relative name
    log_print(f"Running: {' '.join(cmd)}")
    proc = subprocess.run(cmd, cwd=HINGEPROT_DIR, capture_output=True, text=True)

    if proc.stdout.strip():
        log_print(proc.stdout.rstrip())
    if proc.stderr.strip():
        log_print(proc.stderr.rstrip())

    return proc.returncode, out_dir_in_hp, out_dir_name

# ---------- UI ----------
css = W.HTML("""
<style>
.hp-card {border:1px solid #e5e7eb; border-radius:14px; padding:14px 16px; margin:10px 0; background:#fff;}
.hp-title {display:flex; align-items:center; gap:10px; margin-bottom:10px;}
.hp-title b {font-size:20px;}
.hp-small {font-size:12px; color:#6b7280; margin-top:6px;}
</style>
""")

header = W.HTML("""
<div class="hp-card">
  <div class="hp-title">
    <span style="display:inline-block;width:14px;height:14px;background:#ef4444;border-radius:999px;"></span>
    <b>HingeProt (Google Colab UI)</b>
  </div>
  <div class="hp-small">Elastic network model hinge prediction (local run in Colab)</div>
</div>
""")

# --- NEW: radio choice ---
input_mode = W.RadioButtons(
    options=[("Enter PDB code", "code"), ("Upload PDB file", "upload")],
    value="code",
    description="Input:",
    style={"description_width": "60px"},
    layout=W.Layout(width="420px")
)

# PDB code entry (starts empty)
pdb_code = W.Text(
    value="",
    description="PDB code:",
    placeholder="e.g., 4cln",
    style={"description_width": "80px"},
    layout=W.Layout(width="420px")
)

# Upload UI (ONE-CLICK uploader; no extra Colab button)
btn_choose_file = W.Button(
    description="Choose file",
    icon="upload",
    layout=W.Layout(width="180px")
)
file_lbl = W.Label("No file chosen")

code_box = W.HBox([pdb_code], layout=W.Layout(align_items="center"))
upload_box = W.HBox([btn_choose_file, file_lbl], layout=W.Layout(align_items="center", gap="10px"))

btn_load = W.Button(
    description="Load / Detect Chains",
    button_style="info",
    icon="search",
    layout=W.Layout(width="260px")
)

chains_select = W.SelectMultiple(
    options=[], description="Select Chains:", rows=8,
    style={"description_width":"110px"},
    layout=W.Layout(width="420px")
)
all_structure = W.Checkbox(value=False, description="All structure")

gnm_cut = W.Dropdown(options=[6.0,8.0,9.0,10.0,11.0,12.0], value=10.0,
                     description="GNM cutoff (Å):", style={"description_width":"120px"})
anm_cut = W.Dropdown(options=[14.0,16.0,18.0,20.0,22.0], value=18.0,
                     description="ANM cutoff (Å):", style={"description_width":"120px"})
email = W.Text(value="", description="E-mail:", placeholder="optional",
               style={"description_width":"80px"}, layout=W.Layout(width="420px"))

progress = W.IntProgress(value=0, min=0, max=1, description="Progress:", bar_style="")
btn_run  = W.Button(description="Submit", button_style="success", icon="play", layout=W.Layout(width="180px"))
btn_clear= W.Button(description="Clear",  button_style="warning", icon="trash", layout=W.Layout(width="180px"))

log_out = W.Output()
hinges_preview = W.Textarea(value="", description="Preview:", layout=W.Layout(width="100%", height="220px"))

state = {
    "pdb_path": None,
    "run_dir": None,
    "upload_name": None,
    "upload_bytes": None,
}

def _show_log(msg: str):
    with log_out:
        print(msg)

def _sync_input_visibility(*_):
    if input_mode.value == "code":
        code_box.layout.display = ""
        upload_box.layout.display = "none"
    else:
        code_box.layout.display = "none"
        upload_box.layout.display = ""
_sync_input_visibility()
input_mode.observe(lambda ch: _sync_input_visibility(), names="value")

# ---- One-click JS uploader (no extra Colab upload widget) ----
def _js_upload_callback(payload):
    """
    payload = {"name": "...", "data_b64": "..."}  (base64 content)
    """
    try:
        name = payload.get("name", "upload.pdb")
        data_b64 = payload.get("data_b64", "")
        if not data_b64:
            _show_log("Upload callback received empty data.")
            return

        data = base64.b64decode(data_b64.encode("utf-8"))

        state["upload_name"] = name
        state["upload_bytes"] = data
        file_lbl.value = name

        _show_log(f"Uploaded file received: {name} ({len(data)} bytes)")
    except Exception as e:
        _show_log(f"Upload callback error: {e}")

# Register callback name used by JS below
output.register_callback("hingeprot_uploader", _js_upload_callback)

def on_choose_file(_):
    """
    Opens file picker immediately. No extra Colab widget/button.
    """
    _show_log("Opening file picker...")

    js = r"""
    (async () => {
      const input = document.createElement('input');
      input.type = 'file';
      input.accept = '.pdb,.ent';
      input.style.display = 'none';
      document.body.appendChild(input);

      input.onchange = async () => {
        const file = input.files && input.files[0];
        document.body.removeChild(input);
        if (!file) return;

        const reader = new FileReader();
        reader.onload = async () => {
          const b64 = (reader.result || "").split(",")[1] || "";
          await google.colab.kernel.invokeFunction(
            "hingeprot_uploader",
            [{name: file.name, data_b64: b64}],
            {}
          );
        };
        reader.readAsDataURL(file);
      };

      input.click();
    })();
    """
    output.eval_js(js)

btn_choose_file.on_click(on_choose_file)

def on_load_clicked(_):
    hinges_preview.value = ""
    with log_out:
        clear_output()

    try:
        ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        run_dir = os.path.join(RUNS_ROOT, f"run_{ts}")
        os.makedirs(run_dir, exist_ok=True)
        state["run_dir"] = run_dir

        pdb_path = os.path.join(run_dir, "input.pdb")

        if input_mode.value == "upload":
            if state["upload_bytes"] is None:
                raise ValueError("Please click 'Choose file' and upload a PDB first.")
            _show_log(f"Saving uploaded PDB: {state['upload_name']}")
            with open(pdb_path, "wb") as f:
                f.write(state["upload_bytes"])
        else:
            code = pdb_code.value.strip()
            if not code:
                raise ValueError("Please enter a PDB code (e.g., 4cln).")
            _show_log(f"Downloading PDB {code} ...")
            _fetch_pdb(code, pdb_path)

        state["pdb_path"] = pdb_path
        _show_log(f"Input PDB saved: {pdb_path} (size={os.path.getsize(pdb_path)} bytes)")

        chs = _detect_chains(pdb_path)
        if not chs:
            raise RuntimeError("No chains detected in the PDB.")
        chains_select.options = chs
        chains_select.value = tuple(chs[:1])

        _show_log(f"Detected chains: {chs}")
        _show_log("Ready. Choose chain(s) or select All structure, then Submit.")
    except Exception as e:
        _show_log(f"ERROR: {e}")

def on_run_clicked(_):
    hinges_preview.value = ""
    with log_out:
        clear_output()

    try:
        if not state["pdb_path"] or not os.path.exists(state["pdb_path"]):
            raise RuntimeError("Please click 'Load / Detect Chains' first.")

        detected = list(chains_select.options)
        if not detected:
            raise RuntimeError("No detected chains. Load again.")

        run_chains = detected if all_structure.value else list(chains_select.value)
        if not run_chains:
            raise RuntimeError("No chain selected.")

        _show_log(f"GNM cutoff selected: {gnm_cut.value} Å")
        _show_log(f"ANM cutoff selected: {anm_cut.value} Å")
        if email.value.strip():
            _show_log(f"E-mail (UI only): {email.value.strip()}")

        _show_log("-" * 70)
        _show_log(f"Running chains: {run_chains}")
        _show_log(f"Working run folder: {state['run_dir']}")
        _show_log("-" * 70)

        progress.max = len(run_chains)
        progress.value = 0
        progress.bar_style = "info"

        collected_dir = os.path.join(state["run_dir"], "results")
        os.makedirs(collected_dir, exist_ok=True)

        for ch in run_chains:
            rc, out_dir_in_hp, out_dir_name = _run_hingeprot_one_chain(state["pdb_path"], ch, _show_log)

            if os.path.isdir(out_dir_in_hp):
                dest = os.path.join(collected_dir, out_dir_name)
                _safe_rm(dest)
                shutil.move(out_dir_in_hp, dest)
                _show_log(f"[{ch}] Output collected -> {dest}")
            else:
                _show_log(f"[{ch}] WARNING: Expected output folder not found: {out_dir_in_hp}")

            if rc != 0:
                _show_log(f"[{ch}] WARNING: Non-zero return code: {rc}")

            progress.value += 1

        progress.bar_style = "success"
        _show_log("-" * 70)

        zip_path = os.path.join(state["run_dir"], "hingeprot_results.zip")
        _zip_folder(collected_dir, zip_path)
        _show_log(f"ZIP created: {zip_path}")

        first_chain = run_chains[0]
        preview_dir = os.path.join(collected_dir, f"input.pdb.{first_chain}")
        hinges_file = None
        if os.path.isdir(preview_dir):
            for fn in os.listdir(preview_dir):
                if fn.endswith(".new.hinges"):
                    hinges_file = os.path.join(preview_dir, fn)
                    break
        hinges_preview.value = _read_small_text(hinges_file) if hinges_file else "(No *.new.hinges found for preview.)"

        files.download(zip_path)

    except Exception as e:
        progress.bar_style = "danger"
        _show_log(f"ERROR: {e}")

def on_clear_clicked(_):
    pdb_code.value = ""
    input_mode.value = "code"
    state["upload_name"] = None
    state["upload_bytes"] = None
    file_lbl.value = "No file chosen"

    chains_select.options = []
    chains_select.value = ()
    all_structure.value = False
    gnm_cut.value = 10.0
    anm_cut.value = 18.0
    email.value = ""

    progress.value = 0
    progress.max = 1
    progress.bar_style = ""

    state["pdb_path"] = None
    state["run_dir"] = None

    with log_out:
        clear_output()
    hinges_preview.value = ""

btn_load.on_click(on_load_clicked)
btn_run.on_click(on_run_clicked)
btn_clear.on_click(on_clear_clicked)

# ----- layout -----
form_card = W.VBox([
    W.HTML('<div class="hp-card">'),
    W.HTML("<b>Input</b>"),
    input_mode,
    code_box,
    upload_box,
    btn_load,
    W.HTML("<hr>"),
    W.HBox([all_structure, chains_select]),
    W.HBox([gnm_cut, anm_cut]),
    email,
    progress,
    W.HBox([btn_run, btn_clear]),
    W.HTML("</div>"),
])

output_card = W.VBox([
    W.HTML('<div class="hp-card"><b>Run Log</b></div>'),
    log_out,
    W.HTML('<div class="hp-card"><b>Hinges Preview</b><div class="hp-small">Shows the beginning of the first chain’s *.new.hinges</div></div>'),
    hinges_preview
])

display(css, header, form_card, output_card)


HTML(value='\n<style>\n.hp-card {border:1px solid #e5e7eb; border-radius:14px; padding:14px 16px; margin:10px …

HTML(value='\n<div class="hp-card">\n  <div class="hp-title">\n    <span style="display:inline-block;width:14p…

Saving 5w6u_trimer.pdb to 5w6u_trimer (2).pdb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>